### Cargado de datos

Este es el codigo con el que se van a cargar los datos, estos datos se van a cargar desde un JSON

In [2]:
import json

with open('datos.json') as file:
    data = json.load(file)

data

{'nodos': [{'nombre': 'Nodo 1',
   'carrera': 'Ingeniería',
   'calle': 'Calle 1',
   'semaforo': True,
   'punto_de_interes': True,
   'conexiones': ['Nodo 2', 'Nodo 3']},
  {'nombre': 'Nodo 2',
   'carrera': 'Medicina',
   'calle': 'Calle 2',
   'semaforo': True,
   'punto_de_interes': False,
   'conexiones': ['Nodo 1', 'Nodo 3']},
  {'nombre': 'Nodo 3',
   'carrera': 'Arquitectura',
   'calle': 'Calle 3',
   'semaforo': False,
   'punto_de_interes': True,
   'conexiones': ['Nodo 1', 'Nodo 2']}]}

### Objeto que se tiene que crear

El objeto Nodo o punto que se tiene que crear y con el metodo para cargar datos y crear un arreglo de objetos

In [3]:
class Nodo:
    def __init__(self, nombre, carrera, calle, semaforo, punto_de_interes, conexiones):
        self.nombre = nombre
        self.carrera = carrera
        self.calle = calle
        self.semaforo = semaforo
        self.punto_de_interes = punto_de_interes
        self.conxiones = conexiones

    def load_nodes(data):
        nodes = []
        for nodo_data in data['nodos']:
            nodo = Nodo(nodo_data['nombre'], nodo_data['carrera'], nodo_data['calle'], nodo_data['semaforo'], nodo_data['punto_de_interes'], nodo_data['conexiones'])
            nodes.append(nodo)
        return nodes


In [4]:
# Este seria el ejemplo de uso para saber si se cargaron bien los datos

# Arreglos de nodos (Ya nuestros datos cargados y convertidos en objetos de datos)
nodes = Nodo.load_nodes(data=data)

# Para verificar que todo cargo bien hacemos un for para verificar dentro del array

for node in nodes:
    print(node.nombre)

Nodo 1
Nodo 2
Nodo 3


In [4]:
import random

taxis = ['carro1', 'carro2', 'carro3', 'carro4']
numero_aleatorio = random.randint(0, 3)
print(numero_aleatorio)
print(taxis[numero_aleatorio])


2
carro3
